In [ ]:
import os
import arcpy
import pandas as pd
import numpy as np
import csv
from arcgis.features import GeoAccessor, GeoSeriesAccessor

arcpy.env.overwriteOutput = True

<h4>Gestión de BD</h4>

In [ ]:
arcpy.env.workspace = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR"
ruta_gdb_inventario = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb"

if arcpy.Exists(ruta_gdb_inventario):
    print("------ La BD SDE PGAR_Inventario Existe ------")
else:
    print("------ Pendiente por definir gestión ------")

<h4>Gestión de Datasets</h4>

In [ ]:
ruta_tbl_tematica = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb\tbl_tematica"
sistema_referencia_inventario = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb\municipio"
arcpy.env.workspace = ruta_gdb_inventario

lista_tematica = []

# Se usa la función SearchCursor para navegar por la tabla alojada en BD
# En este caso no se utiliza un filtro where sql
with arcpy.da.SearchCursor(ruta_tbl_tematica,"nombre_tematica") as cursor:
    for i in cursor:
        if i[0] != '_6_1_Jurisdiccion_CAR':
            lista_tematica.append(i[0])
 
for dataset in lista_tematica:
    ruta_dataset = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb" + "\\" + dataset
    if arcpy.Exists(ruta_dataset):
        print("El Feature DataFrame {0}, existe".format(dataset))
        for capa in arcpy.ListFeatureClasses(feature_dataset=dataset):
            arcpy.Delete_management(capa)
            print(" - Capa {0} borrada".format(capa))
    else:
        arcpy.management.CreateFeatureDataset(ruta_gdb_inventario, dataset, sistema_referencia_inventario)
        print("Se crea el Feature DataFrame {0}".format(dataset))
        
for capa in arcpy.ListFeatureClasses():
    arcpy.Delete_management(capa)
    print(" - Capa {0} borrada".format(capa))
    
for tabla in arcpy.ListTables():
    arcpy.Delete_management(tabla)
    print(" - Tabla {0} borrada".format(tabla))

<h4>Migración de registros .gdb - .sde</h4>

In [ ]:
ruta_origen = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb"
arcpy.env.workspace = ruta_origen
lista_capas_no_exentas = []
lista_capas_exentas = []

# Listar capas NO excluidas de la copia a FDF
for capas_no_exentas in arcpy.ListFeatureClasses():
    if "_" in capas_no_exentas and '_6_1_Jurisdiccion_CAR' not in capas_no_exentas:
        lista_capas_no_exentas.append(capas_no_exentas)
        # print("La capa o tabla {0} NO se migra".format(capas_exentas))
    else: 
        print("La capa {0} se migra".format(capas_no_exentas))

# Listar capas NO excluidas de la copia a FDF
for capas_exentas in arcpy.ListFeatureClasses():
    if "_" not in capas_exentas:
        lista_capas_exentas.append(capas_exentas)
        print("La capa o tabla {0} NO se migra".format(capas_exentas))
    # else: 
    #     print("La capa {0} se migra".format(capas_exentas))          

# Lista de rutas con todas las capas a migrar
ruta_capas = []
for capas in arcpy.ListFeatureClasses():
    ruta_capa = os.path.join(arcpy.env.workspace,capas)
    ruta_capas.append(ruta_capa)
    
# Migración de capas al Feature DataFrame correspondiente
for i in arcpy.ListFeatureClasses():
    for r in lista_capas_no_exentas:
        if (i == r):
            for j in lista_tematica:
                if (i.split("_")[1] == j.split("_")[1]):
                    capa_origen = os.path.join(arcpy.env.workspace, i)
                    capa_destino = os.path.join(ruta_gdb_inventario, j, i)
                    arcpy.management.CopyFeatures(capa_origen, capa_destino)
                    print("La capa {0}, se ha migrado al FDF {1} de la BD GDB".format(i,j))
    for y in lista_capas_exentas:
        if (i == y):
            capa_origen = os.path.join(arcpy.env.workspace, i)
            capa_destino = os.path.join(ruta_gdb_inventario, i)
            arcpy.management.CopyFeatures(capa_origen, capa_destino)
            print("La capa {0} se ha migrado BD GDB".format(i))

# Migración de tablas con prefijo -tbl-
for tablas in arcpy.ListTables():
    if "tbl" in tablas:
        capa_origen = os.path.join(arcpy.env.workspace, tablas)
        capa_destino = os.path.join(ruta_gdb_inventario, tablas)
        arcpy.management.Copy(capa_origen, capa_destino)
        print("La tabla {0} se ha migrado BD GDB".format(tablas))      

<h4>Paramtrización capas que tienen el mismo nombre</h4>
<p>Esto aplica para las capas asociadas a AVR</p>

In [ ]:
arcpy.env.workspace = ruta_gdb_inventario

nombre_dataset = ['_10_Analisis_Vulnerabilidad_Riesgo_Inundacion',
           '_11_Analisis_Vulnerabilidad_Riesgo_Incendio',
           '_12_Analisis_Vulnerabilidad_Riesgo_Avenida_Torrencial',
           '_13_Analisis_Vulnerabilidad_Riesgo_Remocion_Masa']

for dataset in arcpy.ListDatasets():
    for seleccion_dataset in nombre_dataset:
        if dataset == seleccion_dataset:
            for capa in arcpy.ListFeatureClasses(feature_dataset=dataset):
                if '_10_' in capa:
                    arcpy.management.Rename(capa, capa + '_Inundacion')
                elif '_11_' in capa:
                    arcpy.management.Rename(capa, capa + '_Incendio')
                elif '_12_' in capa:
                    arcpy.management.Rename(capa, capa + '_Avenida_Torrencial')
                elif '_13_' in capa:
                    arcpy.management.Rename(capa, capa + '_Remocion_Masa')
                else:
                    print("No se localiza la capa dentro del listado disponible")        
    

<h4>Renombrado de capas</h4>

In [ ]:
for dataset in lista_tematica:
    for capas in arcpy.ListFeatureClasses(feature_dataset=dataset):
        if ("_" in capas[0]) and ("_" in capas[3]): # !! _10_
            arcpy.management.Rename(capas, capas[4:])
            print("Primera modificación exitosa: {0}".format(capas))        
        elif ("_" in capas[0]) and ("_" in capas[2]) and ("_" in capas[4]): # !! _1_4_
            arcpy.management.Rename(capas, capas[5:])
            print("Segunda modificación exitosa: {0}".format(capas))
            print(capas[5:])
        elif ("_" in capas[0]) and ("_" in capas[2]) and ("_" in capas[5]): # !! _1_15_
            arcpy.management.Rename(capas, capas[6:])
            print("Tercera modificación exitosa: {0}".format(capas))
        elif ("_" in capas[0]) and ("_" in capas[3]) and ("_" in capas[5]): # !! _12_1_
            arcpy.management.Rename(capas, capas[6:])
            print("Cuarta modificación exitosa: {0}".format(capas))
        elif ("_" in capas[0]) and ("_" in capas[3]) and ("_" in capas[6]): # !! _10_12_
            arcpy.management.Rename(capas, capas[7:])
            print("Quinta modificación exitosa: {0}".format(capas))
        else:
            print("No se renomabra la capa {0}".format(capas))
            
print("--------------- Finaliza cambio de nombres de las capas ----------------")

<h4>Renombrado de datasets</h4>

In [ ]:
for dataset in arcpy.ListDatasets():
    if ("_" in dataset[0]) and ("_" in dataset[2]): # !! _1_
            arcpy.management.Rename(dataset, dataset[3:])
            print("Primera modificación exitosa: {0}".format(dataset))
    elif ("_" in dataset[0]) and ("_" in dataset[3]): # !! _10_
            arcpy.management.Rename(dataset, dataset[4:])
            print("Segunda modificación exitosa: {0}".format(dataset))
    else:
        print("No se renomabra Feature Dataset {0}".format(dataset))

print("--------------- Finaliza cambio de nombres de Features Datasets ----------------")
    

<h4>Creación de relaciones M:N</h4>

In [ ]:
feature_municipio = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb\municipio"
tabla_capa = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb\tbl_capa"
tabla_cuenca = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb\cuenca"

# ********************** Relación Municipio - Capa **********************
# - Creación de la estrucutura de la relación
r_municipio_capa = "r_municipio_capa"
arcpy.management.CreateRelationshipClass(tabla_capa, feature_municipio, r_municipio_capa, 'SIMPLE', 'capa', 'municipio', 
                                         "NONE", 'MANY_TO_MANY', "NONE", 'codigo_capa', 'codigo_capa', 'codigo_municipio', 'codigo_municipio')
print("Creada relación: {0}".format(r_municipio_capa))

# - Creación de la vista de tabla
ruta_csv_relacion_municipio_capa = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\3.PGAR\Resultados\2.Py\r_municipio_capa.csv"

# - Unificación de registros
arcpy.Append_management(ruta_csv_relacion_municipio_capa, r_municipio_capa, "NO_TEST")
print(" - Se incorporan los registros de la relación: {0}".format(r_municipio_capa))

# ********************** Relación Municipio - Cuenca **********************
# - Creación de la estrucutura de la relación
r_municipio_cuenca = "r_municipio_cuenca"
arcpy.management.CreateRelationshipClass(tabla_cuenca, feature_municipio, r_municipio_cuenca, 'SIMPLE', 'cuenca', 'municipio', 
                                         "NONE", 'MANY_TO_MANY', "NONE", 'codigo_cuenca', 'codigo_cuenca', 'codigo_municipio', 'codigo_municipio')
print("Creada relación: {0}".format(r_municipio_cuenca))

# - Creación de la vista de tabla
ruta_csv_relacion_municipio_cuenca = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\3.PGAR\Resultados\2.Py\r_municipio_cuenca.csv"

# - Unificación de registros
arcpy.Append_management(ruta_csv_relacion_municipio_cuenca, r_municipio_cuenca, "NO_TEST")
print(" - Se incorporan los registros de la relación: {0}".format(r_municipio_cuenca))

<h4>Creación de relaciones N:1</h4>

In [ ]:
tabla_tematica = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb\tbl_tematica"
tabla_direccion = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb\tbl_direccion"
tabla_departamento = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb\tbl_departamento"
tabla_fuente = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb\tbl_fuentes"
r_capa_tematica = "r_capa_tematica"
r_direccion_municipio = "r_direccion_municipio"
r_municipio_departamento = "r_municipio_departamento"
r_capa_fuentes = "r_capa_fuentes"

# ********************** Relación Capa - Temática **********************
arcpy.management.CreateRelationshipClass(tabla_tematica, tabla_capa, r_capa_tematica, 'SIMPLE', 'capa', 'tematica', 
                                         "NONE", 'ONE_TO_MANY', "NONE", 'codigo_tematica', 'codigo_tematica')
print("Creada relación: {0}".format(r_capa_tematica))

# ********************** Relación Dirección - Municipio **********************
arcpy.management.CreateRelationshipClass(tabla_direccion, feature_municipio, r_direccion_municipio, 'SIMPLE', 'municipio', 'direccion', 
                                         "NONE", 'ONE_TO_MANY', "NONE", 'codigo_direccion', 'codigo_direccion')
print("Creada relación: {0}".format(r_direccion_municipio))

# ********************** Relación Municipio - Departamento **********************
arcpy.management.CreateRelationshipClass(tabla_departamento, feature_municipio, r_municipio_departamento, 'SIMPLE', 'municipio', 'departamento', 
                                         "NONE", 'ONE_TO_MANY', "NONE", 'codigo_departamento', 'codigo_departamento')
print("Creada relación: {0}".format(r_municipio_departamento))

# ********************** Relación Capa - Fuentes **********************
arcpy.management.CreateRelationshipClass(tabla_fuente, tabla_capa, r_capa_fuentes, 'SIMPLE', 'capa', 'fuente', 
                                         "NONE", 'ONE_TO_MANY', "NONE", 'codigo_fuente', 'codigo_fuente')
print("Creada relación: {0}".format(r_capa_fuentes))